## Q26 双线性插值 (Bilinear Interpolation)

-----
[参考](https://www.cnblogs.com/sdxk/p/4056223.html)
------

双线性插值考察4邻域的像素点，根据距离设置权值。虽然计算量增大使得处理时间变长，但是可以有效抑制画质劣化


### opencv 函数

```
cv2.resize(src,dsize,dst=None,fx=None,fy=None,interpolation=None)
```
* src 原图
* dsize 输出原图像
* fx 沿着水平轴的比例因子
* fy 沿着垂直轴的比例因子
* interprolation 插值方法

|interpolation 选项|所用的插值方法|
|-|-|
|INTER_NEAREST    |最近邻插值|
|INTER_LINEAR     |双线性插值（默认设置）|
|INTER_AREA      |	使用像素区域关系进行重采样。 它可能是图像抽取的首选方法，因为它会产生无云纹理的结果。 但是当图像缩放时，它类似于INTER_NEAREST方法。|
|INTER_CUBIC     |	4x4像素邻域的双三次插值 |
|INTER_LANCZOS4   |	8x8像素邻域的Lanczos插值 |




In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [58]:
img = cv2.imread("gg.jpg")

(H, W, C) = img.shape
print("H: ", H, "W:", W)

ratio = 1.5

Height = int(H * ratio)
Width = int(W * ratio)
print("after H: ", Height, "after W:", Width)

img_out = np.zeros((Height, Width, C), dtype = np.int)

for height in range(Height):
    for width in range(Width):
        
        h = int(height / ratio)
        w = int(width / ratio)
        # 边界处理
        if w > W - 2:
            w = W - 2
        if h > H - 2:
            h = H - 2
        
#         fR1 = (width / ratio - w) * img[h , w+1] + (w + 1 - (width / ratio)) * img[h, w]
#         fR2 = (width / ratio - w) * img[h + 1 , w+1] + (w + 1 - (width / ratio)) * img[h + 1, w]
#         img_out[height, width] = (height / ratio - h) * fR2 + (h + 1 - (height / ratio)) * fR1
        
        dh = (height / ratio) - h
        dw = (width / ratio) - w
#         print(dh, dw)
        img_out[height, width] = dh* dw * img[h + 1, w + 1] + (1-dh) * dw * img[h, w + 1] + \
                                dh * (1- dw) * img[h+1, w] + (1- dh) * (1- dw) * img[h + 1, w + 1]

img_out = img_out.astype(np.uint8)

# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img_out = cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB)
            
cv2.imshow("img", img)
cv2.imshow("img_out", img_out)
cv2.waitKey()


H:  128 W: 128
after H:  192 after W: 192


-1

## 双三次插值Bicubic Interpolation

[参考](https://blog.csdn.net/linqianbi/article/details/78594019)

**代码有误**

In [57]:
img = cv2.imread("gg.jpg")

(H, W, C) = img.shape

ratio = 1.5

Height = int(H * ratio)
Width = int(W * ratio)

img_out = np.zeros((Height, Width, C), dtype=np.int)
d = np.zeros((4, 4), dtype= np.int)

# 构造BiCubic基函数
def BiCubic(dis):
    a = -0.5
    ab_dis = np.abs(dis)
    if ab_dis <= 1:
        weight = (a + 2) * np.power(ab_dis, 3) - (a + 3) * np.power(ab_dis, 2) + 1
    elif (ab_dis < 2):
        weight = a* np.power(ab_dis, 3) - 5 * a * np.power(ab_dis, 2) + 8 * a * ab_dis - 4 * a
    else:
        weight = 0
    return weight
    
img_out = np.zeros((Height, Width, C), dtype = np.int) 

for height in range(Height):
    for width in range(Width):
        for c in range(C):
            h = int(height / ratio)
            w = int(width / ratio)
#             print(h , w)
            
            # 边界处理
            if w > W - 3:
                w = W - 3
            if h > H - 3:
                h = H - 3
            if h < 1:
                h = 1
            if w < 1:
                w = 1

            dh = (height / ratio) - h
            dw = (width / ratio) - w

#             h_list = np.transpose(np.array([[dh+1, dh, 1-dh, 2-dh]], dtype=np.float32))
#             w_list = np.array([[dw+1, dw, 1-dw, 2-dw]], dtype=np.float32)
#             h_list = [BiCubic(h_list[i][0]) for i in range(len(h_list))]
#             w_list = [BiCubic(w_list[0][i]) for i in range(len(w_list[0]))]
            w_list = np.transpose(np.array([[dw+1, dw, 1-dw, 2-dw]], dtype=np.float32))
            h_list = np.array([[dh+1, dh, 1-dh, 2-dh]], dtype=np.float32)
            w_list = [BiCubic(w_list[i][0]) for i in range(len(w_list))]
            h_list = [BiCubic(h_list[0][i]) for i in range(len(h_list[0]))]

    #         print(h_list)
    #         print(w_list)

            dis = np.dot(h_list, w_list)
#             dis = [[BiCubic(dis[i][j]) for j in range(dis.shape[1])] for i in range(dis.shape[0])]
            
#             print("img shape",img[h - 1 : h + 3, w - 1 : w + 3, c].shape)
#             print("dis shape", dis.shape)
            img_out[height, width, c] = np.sum(img[h - 1 : h + 3, w - 1 : w + 3, c] * dis)

img_out = img_out.astype(np.uint8)

cv2.imshow("img", img)
cv2.imshow("img_out", img_out)
cv2.waitKey()        

-1